In [1]:
!pip install opencv-python numpy langchain

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)

   ------ ---------------------------------  2/12 [ormsgpack]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-python]
   ------------- --------------------------  4/12 [opencv-py

In [4]:
!pip install langchain-openai

   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   --------------------------------------- 879.1/879.1 kB 19.1 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [tiktoken]
   ---------------------------------------- 0/2 [tiktoken]
   -------------------- ------------------- 1/2 [langchain-openai]
   -------------------- ------------------- 1/2 [langchain-openai]
   -------------------- ------------------- 1/2 [langchain-openai]
   -------------------- ------------------- 1/2 [langchain-openai]
   ---------------------------------------- 2/2 [langchain-openai]



In [11]:
import cv2
import json
import os
import operator
from typing import Annotated, List, TypedDict
from collections import Counter
from dotenv import load_dotenv

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, ToolMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages  # <--- CRITICAL IMPORT

In [13]:
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Check your .env file for OPENAI_API_KEY")

In [14]:
@tool
def check_dominant_colors(image_path: str):
    """
    Analyzes the image to find the top 3 dominant colors in Hex format.
    Use this when a rule requires checking specific colors (e.g. #FF0000).
    Returns: List of hex strings like ['#ff0000', '#ffffff'].
    """
    print(f"\n   [TOOL ACTIVITY] 🛠️  Opening image: {image_path}...")
    
    if not os.path.exists(image_path):
        return "Error: Image file not found."

    image = cv2.imread(image_path)
    if image is None: 
        return "Error: Could not read image."

    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Fast color extraction
    pixels = image.reshape(-1, 3)
    counts = Counter(map(tuple, pixels[::10])) 
    top_3 = counts.most_common(3)
    
    hex_colors = []
    for color, count in top_3:
        hex_str = '#{:02x}{:02x}{:02x}'.format(*color)
        hex_colors.append(hex_str)
        
    print(f"   [TOOL ACTIVITY] 🔍  Found Colors: {hex_colors}\n")
    return hex_colors

In [15]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize the LLM with the Tool
tools = [check_dominant_colors]
llm = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)

# Node 1: The Brain
def call_model(state: AgentState):
    messages = state["messages"]
    response = llm.invoke(messages)
    # We return a list, and LangGraph will append it to the existing state
    return {"messages": [response]}

# Node 2: The Tool Runner
tool_node = ToolNode(tools)

# The Workflow Graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

# Conditional Edge
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [16]:
def run_compliance_check(image_path, json_rules_path):
    # Ensure dummy rule file exists if not provided
    if not os.path.exists(json_rules_path):
        print(f"Creating dummy {json_rules_path}...")
        with open(json_rules_path, "w") as f:
            f.write("""{"rules": [{"rule_id": "test", "description": "Ball must be red", "parameters": {"hex": "#FF0000"}}]}""")

    with open(json_rules_path, "r") as f:
        rules = f.read()

    print(f"--- 🚀 STARTING TEST ON {image_path} ---")
    
    initial_message = HumanMessage(
        content=f"""
        You are a Compliance Officer. 
        1. Read these rules: {rules}
        2. Check this image: '{image_path}'
        3. VERIFY if the image follows the color rule. 
        4. YOU MUST USE THE TOOL to see the colors. Do not guess.
        """
    )
    
    inputs = {"messages": [initial_message]}

    try:
        for event in app.stream(inputs):
            for key, value in event.items():
                if key == "agent":
                    msg = value["messages"][0]
                    print("🤖 [AGENT THOUGHT]:")
                    if msg.tool_calls:
                        print(f"    I need to check the color. Calling tool: {msg.tool_calls[0]['name']}")
                    else:
                        print(f"    Final Answer: {msg.content}")
                elif key == "tools":
                    print("⚡ [TOOL RESULT]:")
                    # In the new State, 'value' might contain the message update
                    # We just print confirmation here
                    print(f"    Tool returned data to the Agent.")
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        print("Tip: Make sure you have the 'langgraph' and 'langchain-openai' libraries updated.")

In [18]:
if __name__ == "__main__":
    # Ensure you have a 'ball.jpeg' (or png) in the folder
    # If not, create a dummy file or use a real path
    if not os.path.exists("ball.jpeg"):
         print("⚠️  Warning: 'ball.jpeg' not found. Please put a red image in this folder named 'ball.jpeg'.")
    else:
         run_compliance_check("ball.jpeg", "test_rules.json")

--- 🚀 STARTING TEST ON ball.jpeg ---
🤖 [AGENT THOUGHT]:
    I need to check the color. Calling tool: check_dominant_colors

   [TOOL ACTIVITY] 🛠️  Opening image: ball.jpeg...
   [TOOL ACTIVITY] 🔍  Found Colors: ['#ffffff', '#fffeff', '#813a40']

⚡ [TOOL RESULT]:
    Tool returned data to the Agent.
🤖 [AGENT THOUGHT]:
    Final Answer: The dominant colors detected in the image 'ball.jpeg' are:

1. #ffffff (White)
2. #fffeff (Almost White)
3. #813a40 (A shade of red, possibly maroon or crimson)

To verify compliance with the rule, we need to check if the primary object (ball) is semantically red. The color #813a40 appears to be a shade of red, which aligns with the rule's requirement for the ball to be semantically red, including shades like crimson or maroon.

Therefore, the image complies with the "ball_color_check" rule, as the detected color #813a40 is recognized as a shade of red.
